In [ ]:
from sklearn import tree
from sklearn.datasets import load_wine
from sklearn.model_selection import train_test_split
import pandas as pd
import graphviz
import matplotlib.pyplot as plt

wine = load_wine()
pd.concat([pd.DataFrame(wine.data), pd.DataFrame(wine.target)], axis=1)


In [ ]:
x_train, x_valid, y_train, y_valid = train_test_split(wine.data, wine.target, test_size=0.3)
clf = tree.DecisionTreeClassifier(criterion="entropy")
clf.fit(x_train, y_train)
score = clf.score(x_valid, y_valid)
score

feature_name = ['酒精', '苹果酸', '灰', '灰的碱性', '镁', '总酚', '类黄素', '非黄烷类分类', '花青素', '颜色强度', '色调', '稀释葡萄酒', '脯氨酸']
dot_data = tree.export_graphviz(clf
                                ,feature_names=feature_name
                                ,class_names=["琴酒", "雪梨", "贝尔摩德"]
                                ,filled=True
                                ,rounded=True
                                )
graph = graphviz.Source(dot_data)
graph


In [ ]:
clf.feature_importances_
[*zip(feature_name, clf.feature_importances_)]

In [ ]:
clf = tree.DecisionTreeClassifier(criterion="entropy", random_state=30)
clf = clf.fit(x_train, y_train)
score = clf.score(x_valid, y_valid)
score
clf = tree.DecisionTreeClassifier(criterion="entropy"
                                    ,random_state=30
                                    ,splitter="random"
                                    ,max_depth=3
                                    ,min_samples_leaf=10
                                    ,min_samples_split=10
                                    )
clf = clf.fit(x_train, y_train)
score = clf.score(x_valid, y_valid)
print(score)
dot_data = tree.export_graphviz(clf
                                ,feature_names=feature_name
                                ,class_names=["琴酒", "雪梨", "贝尔摩德"]
                                ,filled=True
                                ,rounded=True
                                )
graph = graphviz.Source(dot_data)
graph



In [ ]:
test = []
for i in range(10):
    clf = tree.DecisionTreeClassifier(max_depth=i+1
                                        ,criterion="entropy"
                                        ,random_state=30
                                        ,splitter="random"
    )
    clf = clf.fit(x_train, y_train)
    score = clf.score(x_valid, y_valid)
    test.append(score)

plt.plot(range(1, 11), test, color="red", label="max_depth")
plt.legend()
plt.show()

from sklearn.datasets import load_boston
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeRegressor

boston = load_boston()
regressor = DecisionTreeRegressor(random_state=0)
# 交叉验证
#cross_val_score(regressor, boston.data, boston.target, cv=10) r^2
cross_val_score(regressor, boston.data, boston.target, cv=10
                ,scoring="neg_mean_squared_error")


In [ ]:
# 实例回归一维数据
import numpy as np
from sklearn.tree import DecisionTreeRegressor
import matplotlib.pyplot as plt
# 生成随机数种子，是随机稳定
rng = np.random.RandomState(1)
# 生成0-1,且不能为一维,生成的是二维的80*1矩阵
#5 * rng.rand(80, 1)
X = np.sort(5 * rng.rand(80, 1), axis=0)
# y只能有一维数据，ravel（）降维n维变成n-1维。一维数组不分行列
y = np.sin(X).ravel()
#plt.figure()
#plt.scatter(X, y, s=20, edgecolors="black", c="darkorange", label="data")
#plt.legend()
#不可能搞到完全相同的数据，需要有一些噪声，通过给每个数字加上随机数
#np.romdom.rand(数据结构)随机生成随机数
y[::5] += 3 * (0.5 - rng.rand(16))
plt.figure()
plt.scatter(X, y, s=20, edgecolors="black", c="darkorange", label="data")
plt.legend()